# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117103e+02     1.524990e+00
 * time: 0.48702096939086914
     1     9.508817e+00     9.107963e-01
 * time: 1.8572759628295898
     2    -1.152404e+01     1.068910e+00
 * time: 1.9325969219207764
     3    -3.442661e+01     8.060679e-01
 * time: 2.036226987838745
     4    -4.755594e+01     5.667253e-01
 * time: 2.1654138565063477
     5    -5.702074e+01     1.699592e-01
 * time: 2.2505879402160645
     6    -5.987860e+01     1.304607e-01
 * time: 2.314117908477783
     7    -6.096622e+01     5.642782e-02
 * time: 2.377722978591919
     8    -6.139924e+01     4.677515e-02
 * time: 2.448861837387085
     9    -6.163582e+01     4.035564e-02
 * time: 2.5305538177490234
    10    -6.183212e+01     3.036949e-02
 * time: 2.594031810760498
    11    -6.196840e+01     2.612459e-02
 * time: 2.6575047969818115
    12    -6.204363e+01     1.705716e-02
 * time: 2.721946954727173
    13    -6.210246e+01     1.329130e-02
 * time: 2.786146879196167

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671062
    AtomicLocal         -18.8557745
    AtomicNonlocal      14.8522715
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336820

    total               -62.261666455540